In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.utils import shuffle
import numpy as np


In [21]:
# CSIC Dataset

df = pd.read_csv('http_requests.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')


In [22]:
# Shuffling of data and filling in na
# Converting to string for use in CV

df = shuffle(df)
df['payload'] = df['payload'].fillna(0)
df['payload'] = df['payload'].apply(lambda x: str(x))
df.drop(columns=['index', 'method', 'url', 'protocol', 'userAgent', 'pragma', 'cacheControl', 'accept', 'acceptEncoding', 'acceptCharset', 'acceptLanguage', 'host', 'connection', 'contentLength', 'contentType', 'cookie'], inplace=True)


                                       payload label
216538                               B1=Entrar  norm
85474   email=olteanu%40elcarritodelacompra.rw  anom
20626                                 cp=09471  anom
187055                                    id=1  norm
90989                            modo=insertar  anom


In [23]:
print(df.head())


                                       payload label
216538                               B1=Entrar  norm
85474   email=olteanu%40elcarritodelacompra.rw  anom
20626                                 cp=09471  anom
187055                                    id=1  norm
90989                            modo=insertar  anom


In [31]:
# Function to generate a 255 dim feature vector from 'payload'
# Return a list of length 255 with features of payloads mapped to it
# Each list index represents an ASCII character and value represents frequency
# q = [4, 5, 2, 6, 12,  ....]


def vectorize_payload(payload):
    vec_255 = [0]*255
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 1))
    vectorized = vectorizer.fit_transform([payload])
    mapped = list(zip(vectorizer.get_feature_names(), vectorized.sum(0).getA1()))
    for x in mapped:
        vec_255[x[1]] += 1
    return vec_255


In [ ]:
# conversion to 256 dimensional vector
vectors = np.array(df['payload'].apply(lambda x: vectorize_payload(x)))
vectors


In [ ]:
print("%d bytes" % (vectors.size * vectors.itemsize))
